In [11]:
pip install jupyter_dash dash dash_core_components dash_html_components dash_daq plotly pandas

Note: you may need to restart the kernel to use updated packages.


In [12]:
import dash_table
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

Load the CSV file into a DataFrame

In [13]:
filtered_cardiac_interventions = pd.read_csv('filtered_cardiac_interventions_ROC.csv')
# Check for any None values and handle them
filtered_cardiac_interventions.dropna(subset=['Latitude Intervention', 'Longitude Intervention', 'deltaT', 'Postal Code'], inplace=True)

filtered_cardiac_interventions.head(5)

,Mission ID,Service Name,Latitude Permanence,Longitude Permanence,Latitude Intervention,Longitude Intervention,Vector Type,T0,T1,T2,...,T5,T6,Abandon Reason,Original File,T3_T0_difference_minutes,Dead,Postal Code,T Max (V+D),T Max (V),deltaT
0,10221520047,HA UR GEEL AZ St Dimpna,51.16013,4.99975,51.20685,5.13662,MUG,2022-06-01 06:45:53,2022-06-01 06:48:30,2022-06-01 06:52:06,...,NaN,2022-06-01 06:49:27,NaN,interventions_cardiac,18.883333,0,2400,8.660256,6.984078,11.899256
1,10221520095,BA ANTW [Kaai 606] SIHWA,51.31287,4.32425,51.24848,4.43869,AMBMUG,2022-06-01 09:28:15,2022-06-01 09:29:42,2022-06-01 09:30:28,...,NaN,2022-06-01 09:59:48,Overleden,interventions_cardiac,4.233333,1,2170,8.660256,6.984078,0.000000
2,10221520224,HA UR ANTW Stuivenberg,51.22249,4.43629,51.28273,4.42134,AMBMUG,2022-06-01 15:11:15,2022-06-01 15:12:22,2022-06-01 15:14:47,...,NaN,2022-06-01 15:43:14,Overleden,interventions_cardiac,13.416667,1,2180,8.660256,6.984078,6.432589
3,10221520316,FA HVP ANTW [Zuid] Antwerpen,51.20550,4.39722,51.17895,4.39024,AMBMUG,2022-06-01 19:22:57,2022-06-01 19:24:31,2022-06-01 19:28:39,...,2022-06-01 20:12:05,2022-06-01 20:28:04,NaN,interventions_cardiac,10.950000,0,2610,8.660256,6.984078,3.965922
4,10221520324,HA UR HERA AZ,51.18513,4.83741,51.20520,4.76586,AMBMUG,2022-06-01 19:45:13,2022-06-01 19:46:14,2022-06-01 19:50:10,...,NaN,NaN,Overleden,interventions_cardiac,9.616667,1,2290,8.660256,6.984078,2.632589


Now we import a .csv file with the list of Belgian Communes by Postal Code, and we add information about their Province and Region

In [14]:
# URL to the raw CSV file
url = "https://github.com/jief/zipcode-belgium/raw/master/zipcode-belgium.csv"

# Column names
column_names = ['Postal Code', 'Commune Name', 'Longitude Commune', 'Latitude Commune']

# Read the CSV file into a DataFrame without specifying a header
communes = pd.read_csv(url, header=None)

# Manually set the column names
communes.columns = column_names

# Ensure no empty spaces and reset index
communes['Commune Name'] = communes['Commune Name'].str.strip()
communes = communes.reset_index(drop=True)

# Define the function to determine the province
def determine_province(postal_code):
    if 2000 <= postal_code <= 2999:
        return "Antwerp"
    elif 1000 <= postal_code <= 1299:
        return "Brussels"
    elif 9000 <= postal_code <= 9999:
        return "East Flanders"
    elif (1500 <= postal_code <= 1999) or (3000 <= postal_code <= 3499):
        return "Flemish Brabant"
    elif (6000 <= postal_code <= 6599) or (7000 <= postal_code <= 7999):
        return "Hainaut"
    elif 4000 <= postal_code <= 4999:
        return "Liege"
    elif 3500 <= postal_code <= 3999:
        return "Limburg"
    elif 6600 <= postal_code <= 7000:
        return "Luxembourg"
    elif 5000 <= postal_code <= 5999:
        return "Namur"
    elif 1300 <= postal_code <= 1499:
        return "Walloon Brabant"
    elif 8000 <= postal_code <= 8999:
        return "West Flanders"
    else:
        return "Unknown"

# Define the function to determine the region
def determine_region(postal_code):
    if 1000 <= postal_code <= 1299:
        return "Brussels"
    elif (1500 <= postal_code <= 3999) or (8000 <= postal_code <= 9999):
        return "Flanders"
    else:
        return "Wallonia"

# Convert Postal Code to int for correct processing
communes['Postal Code'] = communes['Postal Code'].astype(int)

# Group by 'Postal Code' and aggregate
communes_aggregated = communes.groupby('Postal Code').agg({
    'Commune Name': lambda x: ' / '.join(x),
    'Longitude Commune': 'mean',
    'Latitude Commune': 'mean'
}).reset_index()

# Apply the functions to create the new columns on the aggregated DataFrame
communes_aggregated['Province'] = communes_aggregated['Postal Code'].apply(determine_province)
communes_aggregated['Region'] = communes_aggregated['Postal Code'].apply(determine_region)

# Save to CSV (optional)
communes_aggregated.to_csv('communes_list.csv', index=False)

# Display the first few rows of the aggregated dataframe
print(communes_aggregated.head(10))

   Postal Code           Commune Name  Longitude Commune  Latitude Commune  \
0         1000              Bruxelles           4.351697         50.846557   
1         1020                 Laeken           4.348713         50.883392   
2         1030             Schaerbeek           4.373712         50.867604   
3         1040              Etterbeek           4.389510         50.836851   
4         1050                Ixelles           4.381571         50.822285   
5         1060           Saint-Gilles           4.345668         50.826741   
6         1070             Anderlecht           4.312340         50.838141   
7         1080   Molenbeek-Saint-Jean           4.322778         50.854355   
8         1081             Koekelberg           4.325708         50.862263   
9         1082  Berchem-Sainte-Agathe           4.292702         50.863984   

   Province    Region  
0  Brussels  Brussels  
1  Brussels  Brussels  
2  Brussels  Brussels  
3  Brussels  Brussels  
4  Brussels  Brussels

Calculate the averages per postal code

In [15]:
# Calculate the average value per postal code
averages = filtered_cardiac_interventions.groupby('Postal Code').agg({
    'deltaT': 'mean',
    'Postal Code': 'size'
}).rename(columns={'Postal Code': 'count'}).reset_index()

# Merge with the communes DataFrame to add the additional columns
averages = averages.merge(communes_aggregated, on='Postal Code', how='left')

# Determine the max value of deltaT for color range
max_deltaT = averages['deltaT'].max()

averages.head(10)

,Postal Code,deltaT,count,Commune Name,Longitude Commune,Latitude Commune,Province,Region
0,1000,2.108992,7,Bruxelles,4.351697,50.846557,Brussels,Brussels
1,1020,3.915922,5,Laeken,4.348713,50.883392,Brussels,Brussels
2,1030,14.032682,8,Schaerbeek,4.373712,50.867604,Brussels,Brussels
3,1040,3.190922,2,Etterbeek,4.389510,50.836851,Brussels,Brussels
4,1060,2.165922,3,Saint-Gilles,4.345668,50.826741,Brussels,Brussels
5,1070,21.634970,7,Anderlecht,4.312340,50.838141,Brussels,Brussels
6,1080,26.782589,4,Molenbeek-Saint-Jean,4.322778,50.854355,Brussels,Brussels
7,1081,10.315922,1,Koekelberg,4.325708,50.862263,Brussels,Brussels
8,1082,5.574256,2,Berchem-Sainte-Agathe,4.292702,50.863984,Brussels,Brussels
9,1083,4.479256,5,Ganshoren,4.317510,50.871240,Brussels,Brussels


Now we create the app

In [16]:
app = JupyterDash(__name__)
app.title = "Cardiac Interventions Rankings"

c:\Users\MOGLIANA\AppData\Local\anaconda3\Lib\site-packages\dash\dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Manually scale the size of the scatters

In [17]:
# Manually scale sizes between 50 and 200
size_min = 20
size_max = 300
count_min = averages['count'].min()
count_max = averages['count'].max()

# Scale the count values to the desired size range
if count_max != count_min:
    averages['scaled_size'] = ((averages['count'] - count_min) / (count_max - count_min)) * (size_max - size_min) + size_min
else:
    averages['scaled_size'] = size_min

Now, it's time to create the map

In [18]:
# Custom color scale from green to red
color_scale = [
    [0, "green"],
    [1, "red"]
]

# Map creation function
def create_figure(df):
        
        hover_texts = df['Commune Name'].apply(lambda x: (str(x)[:30] + '...') if isinstance(x, str) and len(x) > 30 else str(x))
        # Format numbers to two decimal places
        df['deltaT'] = df['deltaT'].round(2)

        fig = px.scatter_mapbox(
            df,
            lat='Latitude Commune',
            lon='Longitude Commune',
            size='count',
            color='deltaT',
            color_continuous_scale=color_scale,
            range_color=[0, 20],
            hover_name=hover_texts,
            hover_data={'Latitude Commune': False, 'Longitude Commune': False, 'count': True, 'deltaT': True},
            labels={'deltaT': 'Time Delay', 'count':'Number of Observations'}
        )

        # Update marker sizes
        fig.update_traces(marker=dict(size=df['scaled_size']))

        fig.update_layout(
            mapbox_style='open-street-map',
            mapbox_zoom=6.8,
            mapbox_center={'lat': 50.8503, 'lon': 4.3517},
            font=dict(family="Arial")
        )
        return fig

# Create initial map figure
fig = create_figure(averages)

Define the Layout

In [19]:
n = 5
k = 10

# Filter out provinces with `None` value to fix the dropdown error
valid_provinces = averages['Province'].dropna().unique()

# Format numbers to two decimal places
averages['deltaT'] = averages['deltaT'].round(2)

# App layout
# App layout
app.layout = html.Div([
    html.H1('Intervention time delay* in case of cardiac arrest, average time per Commune', style={'font-family': 'Arial', 'text-align': 'center'}),
    html.H2('Interactive Map and Table (showing the Communes with the highest intervention time delay*), also by Province', style={'font-family': 'Arial', 'text-align': 'center'}),
    dcc.Dropdown(
        id='province-filter',
        options=[{'label': province, 'value': province} for province in valid_provinces],
        multi=True,
        placeholder="Select Province(s)",
        style={'font-family': 'Arial'}
    ),
    html.Div([
        dcc.Graph(id='map', figure=fig, style={'width': '65%', 'display': 'inline-block', 'height': '600px'}),
        dash_table.DataTable(
            id='table',
            columns=[
                {'name': 'Postal Code', 'id': 'Postal Code'},
                {'name': 'Commune Name(s)', 'id': 'Commune Name'},
                {'name': 'Time Delay', 'id': 'deltaT'},
                {'name': 'Number of Observations', 'id': 'count'},
                {'name': 'Province', 'id':'Province'}
            ],
            style_table={'margin-top': '50px', 'width': '100%', 'height': '600px', 'overflowY': 'auto'},
            style_cell={'whiteSpace': 'normal','height': 'auto', 'font-family': 'Arial', 'font-size': '14px'},
            page_size=k,  # Ensure that the table can hold at least k rows
        )
    ], style={'display': 'flex'}),
    html.P("* 'Delay' refers to the difference between the average historical time of intervention in the commune and a time of intervention (statistically individuated) which ensures a good chance of survival. For further information, please consult the documentation.", style={'font-family': 'Arial', 'margin-top': '10px', 'text-align': 'center'})
])

# Callbacks for interactions
@app.callback(
        [Output('map', 'figure'), Output('table', 'data')],
        [Input('province-filter', 'value')]
    )
def update_map_and_table(selected_provinces):
        if selected_provinces:
            filtered_df = averages[averages['Province'].isin(selected_provinces)]
        else:
            filtered_df = averages

        filtered_df_min_n = filtered_df[filtered_df['count'] >= n]
        top_k_communes = filtered_df_min_n.nlargest(k, 'deltaT')

        # Truncate the Commune Name for display in the table
        top_k_communes['Commune Name'] = top_k_communes['Commune Name'].apply(lambda x: (x[:30] + '...') if len(x) > 30 else x)
        
        # Update map
        fig = create_figure(filtered_df)

        # Center map on selected provinces with a zoom (set to average center here for simplicity)
        if selected_provinces:
            center_lat = filtered_df['Latitude Commune'].mean()
            center_lon = filtered_df['Longitude Commune'].mean()
            fig.update_layout(mapbox_center={'lat': center_lat, 'lon': center_lon}, mapbox_zoom=7.5)
        else:
            fig.update_layout(mapbox_center={'lat': 50.8503, 'lon': 4.3517}, mapbox_zoom=6.8)

        return fig, top_k_communes.to_dict('records')

Run the app

In [20]:
# Run the app without 'mode'
try:
    app.run_server(debug=True)
    print("App running at http://127.0.0.1:8050/")
except Exception as e:
    print(f"Error running server: {e}")

Error running server: 'NoneType' object cannot be interpreted as an integer
